### Rate Limits

- 20 requests every 1 seconds 
- 100 requests every 2 minutes

### First get 10000 match histories.

- First 10 columns are the champions for blue team, red team and the last columns stands for "Blue team won"

In [28]:
import cassiopeia as cass
import getpass
import random

key = getpass.getpass("Enter Riot API key:")

Enter Riot API key:········


In [29]:
cass.set_riot_api_key(key)  # This overrides the value set in your configuration/settings.
cass.set_default_region("KR")

In [32]:
import pickle as pkl
import os
import pandas as pd
from cassiopeia.datastores.riotapi.common import APIRequestError, APIError
from datapipelines.pipelines import NotFoundError


In [33]:
def get_lane(participant):
    try:
        lane = participant.lane.name
        if lane =='bot_lane':
            lane = participant.role.name
        return lane
    except AttributeError:
        return 'No info'

In [34]:
def get_data(i):
    dd = []
    match = cass.get_match(i)
    try:
        version = match.version
        queue = match.queue.value
    except APIRequestError:
        print('Your API key has expired.')
        key = getpass.getpass("Enter new Riot API key:")
        cass.set_riot_api_key(key)
        try:
            version = match.version
            queue = match.queue.value
        except:
            return []
    except:
        return []
        
    if queue in ['RANKED_SOLO_5x5'] and version == '9.1.257.7563':        
        for participant in match.blue_team.participants :
            try:
                lane = get_lane(participant)
                champion = participant.champion.name
                rank = participant.summoner.ranks[match.queue]
                tier = ' '.join([rank.tier.name, rank.division.name])
            except APIRequestError:
                print('Your API key has expired.')
                key = getpass.getpass("Enter new Riot API key:")
                cass.set_riot_api_key(key)
                lane = get_lane(participant)
                champion = participant.champion.name
                rank = participant.summoner.ranks[match.queue]
                tier = ' '.join([rank.tier.name, rank.division.name])
            
            dd.append((lane, participant.champion.name, tier))
            
        for ban in match.blue_team.bans:
            try:
                dd.append(ban.name)
            except AttributeError:
                dd.append('밴없음')
                
        for participant in match.red_team.participants :
            try:
                lane = get_lane(participant)
                champion = participant.champion.name
                rank = participant.summoner.ranks[match.queue]
                tier = ' '.join([rank.tier.name, rank.division.name])
            except APIRequestError:
                print('Your API key has expired.')
                key = getpass.getpass("Enter new Riot API key:")
                cass.set_riot_api_key(key)
                lane = get_lane(participant)
                champion = participant.champion.name
                rank = participant.summoner.ranks[match.queue]
                tier = ' '.join([rank.tier.name, rank.division.name])
            
            dd.append((lane, participant.champion.name, tier))
            
        for ban in match.red_team.bans:
            try:
                dd.append(ban.name)
            except AttributeError:
                dd.append('밴없음')

        dd.append(match.blue_team.win)
        print('Match id {} is valid.'.format(i))
    return dd

In [67]:
aa = cass.get_featured_matches()

In [81]:
dd = aa[2]

In [82]:
bb = aa[2].participants

In [86]:
cc = bb[1]

In [87]:
cc.summoner.name

'플루토오'

In [66]:
cc.side

<Side.blue: 100>

# Set start_id and end_id

||start_id|end_id|
|----------|------------|------------|
| Jaewoo   | 3490000000 | 3492500000 |
| Jinyoung | 3492500001 | 3495000000 |
| Hwi      | 3495000001 | 3475000000 |
| Q        | 3475000001 | 3500000000 |

In [6]:
start_id = 3492500001
end_id = 3495000000
ids = range(start_id, end_id)

dir_name = '{}-{}'.format(start_id, end_id)
os.mkdir(os.path.join('../data/',dir_name))

data = []
user = os.path.realpath('.').split('/')[2]
k=0

FileExistsError: [Errno 17] File exists: '../data/3492500001-3495000000'

In [ ]:
for i in ids:
    try:
        dd = get_data(i)
        if len(dd)!=0:
            data.append(dd)
        if len(data)==2000:
            print("Making {}.pkl.".format(k))
            with open(os.path.join('../data/', dir_name, '{}.pkl'.format(k)), 'wb') as f:
                pkl.dump(data, f)
            k+=1
            data = []
    except:
        print('Error occured for {}'.format(i))
        continue

if len(data)!=0:
    with open(os.path.join('../data/', dir_name, '{}.pkl'.format(k)), 'wb') as f:
        pkl.dump(data, f)
        

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/wuBpujAox1NaldEXKSYCzFsQAL-WmDphLiXoadOIP1gxK3E
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/B8jjbKmDWmnPb-2l6v-qummCj3LB35TNdqqxRWimaOGjVg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/vjxLH_3DrwkgJ-xm9GJdVsYL9S9ENreIvi9fAhxMa1zDaXE
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/WSqdl06HEZvpP2NSayQb3TXnO47sq7MSxmnS5ozoorg-NA
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/Up5Ky9M_FDGowVujgVsybQ4AmtvsVVqUFaEbVXe0pxGEqA
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/yZbrp_tW-cfnB6pSwBBZm12jQJ2MfJyCkcYhzOGAwGRqRoE
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/YVb1PeW-FDE1hycXwI1TBvv0q5anQyZPHx1KPFBbtLrYvq0
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/Hme-Ko-Sw1fUPMFK4A2WVuw0N7z0dKnh4_6EdYEU6Nc

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/xsyzUqMIuZ5PK2IwUm0ODAUYZ26LsjvWbsd9j53KMrcBbFU
Match id 3492589732 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589733
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589734
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589735
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/4GlJd5WicpefiZUV9G9k62WZrWWtU5SR0jaboXREkgUXw8Y
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/3LevhZXgugge2mvz3msnNlvD_o1LoDjO3zp0EspS83lK5jU
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/0MNUMM-t7vi3IG1ny1758BhkUGtJBUps-67gi_CipM4i2Ng
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/0nI7Fh-f6pBiN1A4ECGwIeUjXCNoawBVu0a0tIlhGjQ-VT4
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/-R-FXXmEgViLfHtQeYOIyDoW-A8_ws78xs8PG2Cg

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/YWNjMaOOiva3vr8_Q193m6BFHXPjYgmWqgZ6RfCsf52EgRE
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/BcLSvAS4B9qZHZQ13AsZURj5EzjzqQc3MLXgg9Fzenm6A14
Match id 3492589757 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589758
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589759
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589760
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589761
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589762
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589763
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589764
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589765
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589766
Making call: https://kr.api.riotgames.com/lol/match/

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/kX43LT8ycUPtFKza9_M240-Ms0div7OB0UrIcjeV7EV_WA4
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/VDAxGMYl807Zaan41uMEoKWYLrvR3h5sN1m3UejT4If1hg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/IP8Ava6qXfsNM7b11SUOur_boFlCr67dLWRAyugCoTjdpg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/rFLYqOQkX2V1zKumtHZnkLK-hHjiT0s_6SXpMnYiZl6Ujg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/yYDPFRkxeTv3zXxYLTdRfp3IgFAGJZlhwCNq_54xJLj1jQ
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/q8IE9kdzT6i91_oike8xtjomZKq8OBs1X0orzRWeV8RZF44
Match id 3492589799 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589800
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/fw339A-O1LEFlwURCe30wp_ANzXfdjyAFr2Ke8ADOaoPVus
Making call: https

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/FSWc0U8MO1ZT_MCwEYhSLihlnc3jFkQGstrsjEWXDfIlXw
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/UBdjorTriX4wOcMFHvJ4T2io88Xn1mqzOMc1uTgCr5SYJ4I
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/6ulOECJEDS7ExTAW886RnXS9HzvLDP9kdr4HoE3aVTW_Uhk
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/FqYUehoUV7ii5GhIjelsEcXsUi0TubBnHfL20D3vpXhQP3o
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/yz4bjMmFHqKTrvpAm_HXtSmmjMfkHTsvlnU6rz_rCqB9Hw
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/bXK5B2C3wYpDJ-4rYx7BwCJe_A39gZW9SIByXO09QkR00WU
Match id 3492589823 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589824
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492589825
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/34925

...

Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590274
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590275
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590276
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590277
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590278
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590279
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/ESqWvaQzA1v-DrkVjt52o5x1w2O9L0F2_5InaDaYeGtVBg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/b-Xmgj8rHSo_83Surbpm-EnXZoWM2QOzJdE4Pc9DG0OqEQ
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/a9jAJYFrUDdj2DhuZb04Go6dFMqq-BlLYioO0clB1AjNCQ
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/YE3xmXg4TdM4DFYc4uz5ep3VvwR8hNmZbkqG360VwvXlub4
Making call: https://kr.api.riotgames.com/lol/league/v4

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/XO9z0ahOvgLRWaILNHxvOtz464yCz6Dr0PL8kYNVjzRT5eI
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/4wr-Dv3zNY_j3XogdJAoq9bLanc8SdqdvM4zWwkieGqQUv0
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/5y869LM9IkwuhDktxm1sExXoRca1H_dAPs5GeHbjpemEBik
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/2y6koDk83iX5Smv74tIffu0-H3bwls-RUJgRslrCl2GCscw
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/sPZzxWhrjGTRcOWj-bLPzw0giq_VqXfqZ91_WOj3Im647PI
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/egh6ykJC_cci4G4XnRZcsBbIPrmSbsldsM09YUza6LsRDn4
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/3WQfmTRs-ew0xlpK_vRr7nJRvaZ3lpulGjFHmgXZOt8UpPc
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/rYwFXLIT0U1w9lvevpFN_tTem7wQbkPhIkP_DZYX

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/lJKPwG8JqaFOGLwGDGX4ugvHfP-GYWy-J4PZdg-bVMTpaFA
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/J_Xk5unTk5Q8TuE9z8D8gZM1TQNOL9hYniI8GYIsuRR9AhM
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/2qXVNl8RXluoBbvBEddJ07yg8sMxbQBQjGyxGP2LnO5maQ
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/xRM4tG-LizpL6ZYhnmuSKM2ktfWVdfiiphV6vHvhUFIB6w
Match id 3492590327 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590328
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590329
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/4SlOR0tqFQwkNmybyhoKwX2j4WFhVhy6Q1hMkaWoQseSPxs
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/8sCEduR19iqtcTnADiB3_wsVVWq0rRP_ZPL97wIxic8m0d4
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/ikhLD3JfasGFbTo-Lx5UbVi-PR2UJqQSxRHUhA9pj3MpQ9s
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/-o6_HHbSR7_D2f5rv1oC2q3gInR2dBt9ZsVrVVyWr4bugQ
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/0ie-oe4Yq_4hhRy9OU1VHFwsBPyX2ImEpm3u9uuM9zOkkA
Match id 3492590363 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590364
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590365
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/XQ6N9VEpag_EiO_GWZrGKvoubRvCEs9jchf3tkJCXw2jKSQ
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/b9AYd_tyr2yOZXQ4yGywdoLA5K7OEvARk5CcNOrWfMQBNQ
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/kf-Lq02Rkp_MdnVy7qW8xY_r_rAey3GUelskXGrMORbDDJ4
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/-HVzRWo-V2LsRfCMhEMCjVj1JWkzm0I-LPUzcP0KICO3bg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/OYBZLG0cPRHQcaPzKufOj4d93aSsI8aHxRW6jBFkXalqKg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/_jPooCeOcaj0hteqV5j5jYFm7SanvwjCE2ni7UAeqUKXWbk
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/8zRxeo-SSNpejWvt_HcvZjMdKPnhsCWbItju_N62Q7U_P9Y
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/pT98rZOSyzwY_3UJCUzZAl2kBtH7vkUlEhrEaHaGUdf02KI
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/A3AlctqXGEIbB1JVtiULh6lBlwd4vwoGhkzVKQrntaB9r0A
Match id 3492590383 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590384
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492590385
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by